AI Agent: Textbook to Notebook Notes Generator 

In [ ]:
!ollama pull llama3:8b
!ollama serve


In [1]:
import os
import json
import gradio as gr
import textwrap
import asyncio
import markdown_it
from dotenv import load_dotenv
from openai import AsyncOpenAI
from pypdf import PdfReader
from pydantic import BaseModel
from fpdf import FPDF

In [2]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key found, starting with: {openai_api_key[:8]}...")
else:
    print("OpenAI API Key not found. Please set it in your .env file.")

if google_api_key:
    print(f"Google API Key found, starting with: {google_api_key[:8]}...")
else:
    print("Google API Key not found. Please set it in your .env file.")


OpenAI API Key found, starting with: sk-proj-...
Google API Key found, starting with: AIzaSyDn...


In [4]:
ollama_client = AsyncOpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

gemini_client = AsyncOpenAI(
    api_key=google_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [5]:
class Evaluation(BaseModel):
    """Defines evaluator output."""
    is_acceptable: bool
    feedback: str

In [6]:
async def generate_notes(text_chunk: str, retries=2, feedback=""):
    """
    Generate structured notes for a given text chunk using Ollama3:8b.
    Self-corrects based on Gemini evaluation feedback.
    """
    system_prompt = (
        "You are an expert academic assistant. "
        "Read the provided text and produce well-organized, clear Markdown notes. "
        "Focus on key concepts, definitions, and main ideas. "
        "Keep the language simple but precise."
    )

    if feedback:
        user_prompt = (
            f"The previous notes were not acceptable. Improve them using this feedback:\n"
            f"{feedback}\n\nOriginal Text:\n{text_chunk}"
        )
    else:
        user_prompt = f"Generate concise academic notes for the following text:\n{text_chunk}"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    try:
        response = await ollama_client.chat.completions.create(
            model="llama3:8b",
            messages=messages,
        )
        notes = response.choices[0].message.content
    except Exception as e:
        print(f"Error during Ollama generation: {e}")
        return f"Error generating notes: {e}"

    if retries > 0:
        evaluation = await evaluate_notes(text_chunk, notes)
        if not evaluation.is_acceptable:
            print(f"Evaluation failed. Retrying with feedback: {evaluation.feedback}")
            return await generate_notes(text_chunk, retries - 1, evaluation.feedback)
        else:
            print("Evaluation passed.")

    return notes

In [ ]:
# MODIFIED: Changed the model name to one compatible with the API endpoint
async def evaluate_notes(text_chunk: str, notes: str) -> Evaluation:
    """
    Evaluates generated notes using Gemini API for accuracy, clarity, and coverage.
    """
    prompt = (
        "You are a quality assurance evaluator. Assess the provided notes "
        "based on their accuracy, clarity, and completeness. "
        "Return a JSON object with two keys only:\n"
        "1. is_acceptable (boolean)\n"
        "2. feedback (string)\n\n"
        f"--- Original Text ---\n{text_chunk}\n\n"
        f"--- Notes ---\n{notes}"
    )

    try:
        response = await gemini_client.chat.completions.create(
            # MODIFICATION IS ON THIS LINE
            model="gemini-flash2.5",
            messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_object"},
        )

        data = json.loads(response.choices[0].message.content)
        return Evaluation(**data)

    except Exception as e:
        print(f"⚠️ Gemini evaluation error: {e}")
        return Evaluation(is_acceptable=True, feedback=f"Evaluation failed: {e}")

In [8]:
def chunk_text(text: str, max_chars: int = 2500):
    """Splits long text into smaller chunks for better processing."""
    return textwrap.wrap(text, width=max_chars, break_long_words=False, replace_whitespace=False)

In [9]:
def create_pdf_file(notes_markdown: str, source_filename: str) -> str:
    """
    Creates a PDF file from the generated Markdown notes using FPDF2.
    """
    title = os.path.splitext(os.path.basename(source_filename))[0].replace('_', ' ').title()
    output_filename = f"{os.path.splitext(source_filename)[0]}_notes.pdf"
    
    # NEW: fpdf2 implementation starts here
    pdf = FPDF()
    pdf.add_page()
    
    # Add a title to the PDF
    pdf.set_font("Arial", "B", 18)
    pdf.cell(0, 10, f"Notes for {title}", 0, 1, "C")
    pdf.ln(10) # Add a little space after the title

    # Set the font for the main content
    # Note: Default fonts have limited character support.
    # For full Unicode, you might need to add a custom font like DejaVu.
    pdf.set_font("Arial", "", 11)
    
    try:
        # Use the experimental write_markdown feature from fpdf2
        # This directly converts the Markdown string into the PDF
        pdf.write_markdown(notes_markdown)
        
        pdf.output(output_filename)
        print(f"PDF created successfully: {output_filename}")
        return output_filename
        
    except Exception as e:
        print(f"Error creating PDF with fpdf2: {e}")
        # Fallback to saving as a markdown file if PDF generation fails
        md_filename = f"{os.path.splitext(source_filename)[0]}_notes.md"
        with open(md_filename, "w", encoding="utf-8") as f:
            f.write(f"# Notes for {title}\n\n{notes_markdown}")
        print(f"As a fallback, Markdown file saved: {md_filename}")
        return md_filename

In [10]:
async def process_textbook(file, progress=gr.Progress()):
    """
    Main orchestrator function. Extracts all text from the PDF, generates
    a single cohesive set of notes, and saves it as a PDF.
    """
    if file is None:
        return None

    pdf_file_path = file.name
    reader = PdfReader(pdf_file_path)
    num_pages = len(reader.pages)
    
    print(f"Extracting text from {num_pages} pages...")
    progress(0, desc="Step 1/3: Extracting Text...")
    full_text = ""
    for i, page in enumerate(reader.pages):
        progress((i + 1) / num_pages, desc=f"Extracting from Page {i + 1}/{num_pages}")
        page_text = page.extract_text()
        if page_text:
            full_text += page_text + "\n"

    if not full_text.strip():
        print("No text could be extracted from the PDF.")
        return None

    print(f"Text extraction complete. Total characters: {len(full_text)}")
    
    chunks = chunk_text(full_text)
    num_chunks = len(chunks)
    all_notes = []
    
    print(f"Generating notes from {num_chunks} text chunks...")
    progress(0, desc="Step 2/3: Generating Notes...")
    for i, chunk in enumerate(chunks):
        progress((i + 1) / num_chunks, desc=f"Processing Chunk {i + 1}/{num_chunks}")
        notes_chunk = await generate_notes(chunk)
        all_notes.append(notes_chunk)
        
    combined_notes = "\n\n---\n\n".join(all_notes)

    progress(1, desc="Step 3/3: Creating PDF...")
    pdf_path = create_pdf_file(combined_notes, pdf_file_path)
    return pdf_path

In [11]:
async def create_notes_interface(file, progress=gr.Progress(track_tqdm=True)):
    if file is not None:
        return await process_textbook(file, progress)
    return "Please upload a textbook to begin."


In [12]:

iface = gr.Interface(
    fn=create_notes_interface,
    inputs=gr.File(label="Upload Textbook (PDF)"),
    outputs=gr.File(label=" Download Generated Notes (.pdf)"),
    title="AI Textbook → PDF Notes Generator",
    description=(
        "Upload a textbook in PDF format. The local llama3:8b model generates a cohesive summary "
        "of the entire book, Gemini API evaluates its quality, and you receive a downloadable PDF."
    ),
)

if __name__ == "__main__":
    iface.launch(server_name="127.0.0.1", share=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Extracting text from 37 pages...
Text extraction complete. Total characters: 50951
Generating notes from 21 text chunks...
⚠️ Gemini evaluation error: Error code: 404 - [{'error': {'code': 404, 'message': 'models/gemini-pro is not found for API version v1main, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.', 'status': 'NOT_FOUND'}}]
Evaluation passed.
⚠️ Gemini evaluation error: Error code: 404 - [{'error': {'code': 404, 'message': 'models/gemini-pro is not found for API version v1main, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.', 'status': 'NOT_FOUND'}}]
Evaluation passed.
⚠️ Gemini evaluation error: Error code: 404 - [{'error': {'code': 404, 'message': 'models/gemini-pro is not found for API version v1main, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.', 'st

C:\Users\kolla\AppData\Local\Temp\ipykernel_17036\1084223039.py:13: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", "B", 18)
C:\Users\kolla\AppData\Local\Temp\ipykernel_17036\1084223039.py:14: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f"Notes for {title}", 0, 1, "C")
C:\Users\kolla\AppData\Local\Temp\ipykernel_17036\1084223039.py:20: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", "", 11)
